# Chatbot Using Deep Learning 
model having 3 layers. First layer 128 neurons, second layer 64 neurons and third output layer

## Step 1. Load the Data

In [30]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import tensorflow
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\melli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\melli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [32]:
intents

{'intents': [{'tag': 'salam',
   'patterns': ['Halo Ara',
    'Halo',
    'Halo',
    'Selamat Pagi',
    'Selamat Siang',
    'Selamat Malam',
    'Selamat Sore',
    'Salam',
    'Hai',
    'Hi',
    'Helo',
    'Hello',
    'Hei',
    'Malam',
    'Pagi',
    'Siang',
    'Sore',
    'Punten',
    'Permisi',
    'Shalom',
    'Assalamualaikum',
    'Helo',
    'Hello',
    'Hallo',
    'Heh',
    'Woy',
    'Ara',
    'Hay',
    'askum',
    'Guys',
    'aloo',
    'met pagi',
    'met siang'],
   'responses': ['Halo! disini Ara untuk ChatBot Pelaporan Pelecehan Seksual. Siapa korban Pelecehan Seksual yang ingin kamu laporkan?',
    'Selamat siang disini Ara untuk ChatBot Pelaporan Pelecehan Seksual. Apakah ada laporan pelecehan seksual?',
    'Selamat pagi disini Ara untuk ChatBot Pelaporan Pelecehan Seksual. Apakah anda akan melaporkan kasus pelecehan seksual yang korbannya orang lain atau anda sendiri?']},
  {'tag': 'laporDiri',
   'patterns': ['iya',
    'betul',
    'gue',
    

## Step 2. Preprocessing the Data

In [33]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [34]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

#print (len(documents), "documents")
#print (len(classes), "classes", classes)
#print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [35]:
# initializing training data
training = []
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
#shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
#create train and test lists, X = patterns, Y = intens
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\melli\AppData\Local\Temp\ipykernel_15200\754323686.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


## Step 3. Define the Model

In [36]:
# create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='relu'))

## Step 4. Compile the Model

In [37]:
#compile model stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay = 1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

## Step 5. Fit the Model

In [38]:
#fitting and saving the model
hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
159/159 [==============================] - 1s 3ms/step - loss: 8.1870 - accuracy: 0.1225
Epoch 2/200
159/159 [==============================] - 0s 2ms/step - loss: 6.0217 - accuracy: 0.1755
Epoch 3/200
159/159 [==============================] - 0s 2ms/step - loss: 4.5604 - accuracy: 0.2601
Epoch 4/200
159/159 [==============================] - 0s 2ms/step - loss: 4.2635 - accuracy: 0.3422
Epoch 5/200
159/159 [==============================] - 0s 2ms/step - loss: 3.9061 - accuracy: 0.4129
Epoch 6/200
159/159 [==============================] - 0s 3ms/step - loss: 3.8652 - accuracy: 0.5051
Epoch 7/200
159/159 [==============================] - 0s 2ms/step - loss: 3.6479 - accuracy: 0.5442
Epoch 8/200
159/159 [==============================] - 0s 2ms/step - loss: 3.5147 - accuracy: 0.6225
Epoch 9/200
159/159 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.4104
Epoch 10/200
159/159 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.

## Step 6. Save the Model

In [39]:
model.save('chatbot_model.h5', hist)

print('model created')

model created


## Step 7. Make the Predictions

In [40]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.load(open('intents.json', 'rb'))
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [41]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [42]:
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\melli\AppData\Local\Programs\Python\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\melli\AppData\Local\Temp\ipykernel_15200\1716917910.py", line 14, in send
    res = chatbot_response(msg)
  File "C:\Users\melli\AppData\Local\Temp\ipykernel_15200\1370899457.py", line 46, in chatbot_response
    res = getResponse(ints, intents)
  File "C:\Users\melli\AppData\Local\Temp\ipykernel_15200\1370899457.py", line 36, in getResponse
    tag = ints[0]['intent']
IndexError: list index out of range
